# A Guided Tour of Ray Core: Remote Classes

[*Remote Classes*](https://docs.ray.io/en/latest/walkthrough.html#remote-classes-actors)
involve using a `@ray.remote` decorator on a class. 

This implements an [*actor*](https://patterns.eecs.berkeley.edu/?page_id=258) pattern, with properties: *stateful*, *message-passing semantics*

---

First, let's start Ray…

In [21]:
import logging
import ray
from random import randint

In [22]:
ray.init(
    ignore_reinit_error=True,
    logging_level=logging.ERROR,
)

{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': '127.0.0.1:6379',
 'object_store_address': '/tmp/ray/session_2021-12-23_12-11-59_289745_7893/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-12-23_12-11-59_289745_7893/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-12-23_12-11-59_289745_7893',
 'metrics_export_port': 57423,
 'node_id': '17cbf202d56a8dc5a06bb9a465075fbd24ce017562e8e0b13059c550'}

## 3. Remote Classes Stateful Actors Pattern

To start, we'll define a class and use the decorator:


Let's use another Actor class and create multiple instances associated with a distinct attribute, such as a name.

In [23]:
@ray.remote
class GoalsScored:
    def __init__ (self, player, age) -> None:
        self._goals = 0
        self._player = player
        self._age = age

    def score (self, goal) -> object:
        self._goals += goal
        return self._goals
       
    def player(self) -> str:
        return self._player
    
    # return three values
    def stats(self) -> object:
        return self._player, self._age, self._goals

Define three Actors: Rolando, Neymar, Messi

In [24]:
%%time 

ronaldo = GoalsScored.remote("Ronaldo", randint(18, 35))
neymar = GoalsScored.remote("Neymar", randint(18, 35))
messi = GoalsScored.remote("Messi", randint(18, 35))

CPU times: user 8.24 ms, sys: 3 ms, total: 11.2 ms
Wall time: 8.13 ms


Update the scores for each player

In [29]:
%%time

ronaldo_ref = ronaldo.score.remote(randint(1, 7))
neymar_ref = neymar.score.remote(randint(1, 7))
messi_ref  = messi.score.remote(randint(1, 7))

CPU times: user 1.09 ms, sys: 481 µs, total: 1.57 ms
Wall time: 1.01 ms


Again, use list comprehension to iterate over each Actor handle instances, along with object_ref for their goals scores, maintained by each distinct actor.

In [30]:
for ref, ref_obj in [(ronaldo, ronaldo_ref), (neymar, neymar_ref), (messi, messi_ref) ]:
   print(f"Player: {ray.get(ref.stats.remote())} and goals scored: {ray.get(ref_obj)}")

Player: ('Ronaldo', 33, 15) and goals scored: 15
Player: ('Neymar', 27, 8) and goals scored: 8
Player: ('Messi', 20, 10) and goals scored: 10


In [ ]:
for ref, ref_obj in [(ronaldo, ronaldo_ref), (neymar, neymar_ref), (messi, messi_ref) ]:
   print(f"Player: {ray.get(ref.player.remote())} and goals scored: {ray.get(ref_obj)}")

Finally, shutdown Ray

In [20]:
ray.shutdown()

---
## References

["A Universal Modular Actor Formalism for Artificial Intelligence"](https://www.ijcai.org/Proceedings/73/Papers/027B.pdf)  
Carl Hewitt, Peter Bishop, Richard Steiger  
*IJCAI* (1973)